# hmmlearn

In [7]:
from hmmlearn import hmm
import numpy as np
import pandas as pd

## 使用的例子
```python
model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.1, 0.8, 0.1])
model.transmat_ = np.array([[0.8, 0.15, 0.05],
                            [0.05, 0.9, 0.05],
                            [0.05, 0.15, 0.8]])
model.means_ = np.array([[0.0], [3.0], [5.0]])
model.covars_ = np.tile(np.identity(1), (3, 1, 1))
X,Z = model.sample(100)
```

### 数据

1. 输入数据是个3维数据，分别表示`value`,`mean`,`std`
2. 隐含层也包含有3个状态，`-1`,`0`,`1`
3. 通过输入，能够获得隐含层状态

In [52]:
## 测试一个CNV数据

pd_data = pd.read_table('./demo_data/CNV.txt',header=-1)
## DataFrame转化为np array 
X = pd_data.iloc[:,2:5].values

print(X)


[[ 0.00332633  0.01462561  0.013982  ]
 [ 0.04076904  0.04572885  0.00896543]
 [ 0.01372047  0.01913687  0.00486392]
 ...
 [-0.00684093 -0.00452812  0.00171942]
 [ 0.14036387  0.10981676  0.08222681]
 [-0.14893568 -0.09466038  0.02201684]]


### 构建模型

In [53]:
## 因为观测值是连续的值，所以使用GaussianHMM

model = hmm.GaussianHMM(n_components=3, covariance_type="full",n_iter=100, init_params="mcs")

## 隐含层的初始状态概率（deletion 0.1，normal 0.8，duplication 0.1），pi
model.startprob_ = np.array([0.1, 0.8, 0.1])

## 状态转移矩阵，A

model.transmat_ = np.array([[0.8, 0.15, 0.05],
                            [0.05, 0.9, 0.05],
                            [0.05, 0.15, 0.8]])


## 因为观测状态为连续的，所以HMM的B矩阵 用高斯分布的参数来描述

model.means_ = np.array([[-1,0,0.05], [0,0,0.05], [1,0,0.05]])
model.covars_ = np.tile(np.identity(3), (3, 1, 1))/100



### 预测

In [56]:
m1 = model.fit(X)
model.score(X)
print(model.predict(X))
X_s, Z = model.sample(10)
print(X_s)
print(Z)

3805.5716927584326
[0 0 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 2 2 1 2 1 0 0 0 1 2 1 2 1 2
 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1
 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 2 1 2 1 2 2 1 2 1
 2 1 2 1 0 0 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 2 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
 1 2 1 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 0 0 0 0 0 0 1 2 1 2 1 2 1 2
 2 1 2 1 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 0 0 0 0 0 1 2 1
 2 1 2 1 2 1 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1
 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2
 1 2 1 2 1 2 1 2 1 2 1 2 1 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1
 2 1 2 1 2 1 2 1 0 0 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2
 1 2 1

/Users/chenyl/.virtualenvs/python-lib-test/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/chenyl/.virtualenvs/python-lib-test/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/chenyl/.virtualenvs/python-lib-test/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=Deprecati